<a href="https://colab.research.google.com/github/stavco9/nlp-final-project/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/stavco9/nlp-final-project.git

Cloning into 'nlp-final-project'...
remote: Enumerating objects: 150, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 150 (delta 59), reused 98 (delta 25), pack-reused 0
Receiving objects: 100% (150/150), 9.03 MiB | 7.03 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [2]:
%cd nlp-final-project/

/content/nlp-final-project


In [3]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  

In [4]:
import os
import json
import pandas as pd
import numpy as np
import requests

In [5]:
# Mount Google Drive
from google.colab import drive, files
drive.mount('/content/drive')
!ln -s "/content/drive/My Drive/IDC_MSc/Year1/NLP/Final-Project/data" ./data

Mounted at /content/drive


In [6]:
# Please set a valid Google API
# Please download the 'data' folder from https://drive.google.com/drive/folders/1D-OHX0LUNA5Y-0h8UXwqa4H_yETZN_t_ to your project root directory

GOOGLE_API_KEY = '**********************'

In [ ]:
##
## Dataset preperation part
##

In [7]:
from modules.dataset.transcript_dataset import TranscriptDataset

dataset_builder = TranscriptDataset(api_key=GOOGLE_API_KEY)

# dataset_folder => The folder to load / save the datasets file from. If not exists it will create it automatically
# videos_count => Total number of videos to include in the dataset. If there are already saved videos in the "data" folder,
# it will fill the remaining videos count (For example if the parameter: videos_count is 4000 and there are already 1500 videos
# in the folder, it will fetch 2500 videos). Default is 100
# country_code => 2 digits ISO Code of the country you want to fetch videos from (If not set, the default is United States 'us')
# Relevant codes for countries with a majority of English based videos => 'ca' Canada, 'uk' United Kingdom, 'au' Australia, 'in' India
#  'nz' New Zealand, 'za' South Africa
# language => 2 digits ISO Code of the video's language you want to use (If not set, the default is English 'en')
dataset_builder.build_dataset(dataset_folder='data/dataset', videos_count=5000, country_code='uk')

# dataset_folder => The folder to load / save the datasets file from. If not exists it will create it automatically
# comments_count => The amount of comments to fetch for each video. Default is 100
dataset_builder.build_comments(dataset_folder='data', comments_count=200)

There are currnet 5000 videos in 10 files
There are already at least 5000. No need to do anything


In [20]:
##
## Sentiment analysis part
##

In [6]:
from modules.sentiment_analysis.comments_sentiment_analysis import multi_sentiment_score
from modules.sentiment_analysis.video_info import Video_info, datumToObj, save_to_json, save_to_pickle
from modules.sentiment_analysis.GPT_comment_controversy import init_openai, get_disagreement_rating
from modules.sentiment_analysis.scoring_video import adjust_disagreement_rating

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
# put all the json into pickle
datasets = [f'data/dataset/dataset_{i}.json' for i in range(10)]
to_print = True #input("Do you want to show prints? (y/n): ") != 'n'
basic_inf_vids = datumToObj(datasets, 'data/sentiment/videos_sample.pkl', to_print)
number_of_vids = len(basic_inf_vids)

save_to_json(basic_inf_vids, 'data/sentiment/vids_no_sentiment.json')
sentiment_model = multi_sentiment_score
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
count = 0
for vid in basic_inf_vids:
    print_text = "\r"
    if to_print:
        count += 1
        print_text += f"Processing video {count}/{number_of_vids}"
    vid.add_comments_scores(sentiment_model, model_name)
    if to_print:
        print_text += f" - Disagreement rating: {vid.comments_controversy_classical}"
    if to_print:
        print(print_text)

save_to_json(basic_inf_vids, 'data/sentiment/vids_classical_sentiment.json')
basic_inf_vids = adjust_disagreement_rating(basic_inf_vids)
save_to_json(basic_inf_vids, 'data/sentiment/videos_classical_sentiment_adjusted.json')
'''

#load vids from videos_classical_sentiment_adjusted.json and adjust basic_inf_vids with its values
with open('data/data/videos_classical_sentiment_adjusted.json') as json_file:
    json_obj = json.load(json_file)
for i, video_obj in enumerate(json_obj):
    basic_inf_vids[i].comments_controversy_classical = float(video_obj['comments_controversy_classical'])
'''


to_continue = True #input("Do you want to continue with GPT? (y/n): ")
if not to_continue:
  print("Done")
else:
  count = 0
  client = init_openai()
  sentiment_model=get_disagreement_rating
  model_name="GPT"
  for vid in basic_inf_vids:
      print_text = "\r"
      if to_print:
          count += 1
          print_text += f"Processing video {count}/{number_of_vids}"
      vid.add_comments_scores(sentiment_model, model_name, client)
      if to_print:
          print_text += f" - Disagreement rating: {vid.comments_controversy_GPT}"
      if to_print:
          print(print_text)
  save_to_json(basic_inf_vids, 'data/data/vids_GPT_sentiment.json')

In [ ]:
##
## Training part
##

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import random
import numpy as np
import json
from modules.azsclm.azsclm import AZSC_LanguageModel
from modules.azsclm.board import Board
from transformers import AutoTokenizer

In [8]:
class Config_train:
    def __init__(self, num_epochs, train_data, model, optimizer, criterion, device):
        self.num_epochs = num_epochs
        self.train_data = train_data
        self.device = device
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion


class Train:
    def __init__(self, config, dataloader, data_name = 'classical', print_level = 0):
        self.num_epochs = config.num_epochs
        self.train_data = config.train_data
        self.device = config.device
        self.model = config.model
        self.optimizer = config.optimizer
        self.criterion = config.criterion
        self.data_name = 'comments_controversy_' + data_name
        self.dataloader = dataloader
        self.board = Board(self, print_level)

    def train(self):
        size = len(self.dataloader.dataset)
        num_batches = len(self.dataloader)
        for epoch in range(self.num_epochs):
            epoch_loss = 0
            total_loss = 0
            for batch_idx, dataloader_part in enumerate(self.dataloader):
                #src, trg = dataloader_part[0].to(self.device), dataloader_part[1].to(self.device)
                #dataloader_part = dataloader_part.to(self.device)
                self.model.train()
                output = self.model(dataloader_part[0])

                # Assuming trg contains the ground truth for the numerical result
                target = dataloader_part[1].to(self.device)

                self.optimizer.zero_grad()
                loss = self.criterion(output, target)
                loss.backward()
                self.optimizer.step()

                total_loss += loss.item()

            avg_loss = total_loss / len(self.dataloader)
            epoch_loss = self.board.info_handler(loss=loss.item(), batch=batch_idx, lenX=len(self.dataloader), size=len(self.dataloader), epoch_loss=epoch_loss, name=self.data_name)

In [9]:
# create a DataLoader objects for training, testing and validation data
# use vids_sentiment.json. the first element of the tuple is the transcript and the second or third element is the score (second element is comments_controversy_GPT and third element is comments_controversy_classical)
# create different data loader for different models

class CustomDataset(Dataset):
    def __init__(self, data, score_method):
        self.data = data
        self.score_method = score_method

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data[idx][0]
        score = self.data[idx][self.score_method]
        return text, score

class Datasets:
    def __init__(self, json_file, ratios=[0.8, 0.1, 0.1], batch_size=32):
        self.transcripts = []
        self.scores = []
        self.json_file = json_file
        assert sum(ratios) == 1
        self.ratios = ratios
        self.batch_size = batch_size
        self.load_data()

    def load_data(self):
        with open(self.json_file, 'r') as f:
            data = json.load(f)

        random.shuffle(data)

        train_size = int(self.ratios[0] * len(data))
        val_size = int(self.ratios[1] * len(data))
        test_size = len(data) - train_size - val_size

        train_data = data[:train_size]
        val_data = data[train_size:train_size+val_size]
        test_data = data[train_size+val_size:]

        train_dataset_classical = CustomDataset(train_data, score_method=1)
        val_dataset_classical = CustomDataset(val_data, score_method=1)
        test_dataset_classical = CustomDataset(test_data, score_method=1)

        train_dataset_gpt = CustomDataset(train_data, score_method=2)
        val_dataset_gpt = CustomDataset(val_data, score_method=2)
        test_dataset_gpt = CustomDataset(test_data, score_method=2)

        # Create data loaders
        self.train_loader_classical = DataLoader(train_dataset_classical, batch_size=self.batch_size, shuffle=True)
        self.val_loader_classical = DataLoader(val_dataset_classical, batch_size=self.batch_size, shuffle=False)
        self.test_loader_classical = DataLoader(test_dataset_classical, batch_size=self.batch_size, shuffle=False)

        self.train_loader_gpt = DataLoader(train_dataset_gpt, batch_size=self.batch_size, shuffle=True)
        self.val_loader_gpt = DataLoader(val_dataset_gpt, batch_size=self.batch_size, shuffle=False)
        self.test_loader_gpt = DataLoader(test_dataset_gpt, batch_size=self.batch_size, shuffle=False)

In [11]:
datasets = Datasets(json_file="data/sentiment/vids_sentiment.json")

tokenizer_name = 'distilbert-base-uncased-finetuned-sst-2-english'
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
tokenizer = lambda x: tokenizer(x, return_tensors="pt", truncation=True, padding=True)

# Instantiate the model
model = AZSC_LanguageModel(tokenizer).to("cuda")

# Instantiate the optimizer
optimizer = optim.Adam(model.parameters(), lr=3e-4)

# Instantiate the loss function
criterion = nn.MSELoss()

config = Config_train(num_epochs=10, train_data=None, model=model, optimizer=optimizer, criterion=criterion, device="cuda")

traind_data = datasets.train_loader_classical
# Instantiate the Train object
trainer = Train(config, traind_data, data_name='classical', print_level=0)

# Start training
trainer.train()

TypeError: <lambda>() got an unexpected keyword argument 'return_tensors'

In [12]:
import torchvision
from torchview import draw_graph

In [13]:
model_graph = draw_graph(trainer.model, input_size=(1,3,224,224), expand_nested=True)
model_graph.visual_graph

RuntimeError: Failed to run torchgraph see error message